In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import hypergeom
from goatools import obo_parser

### Select strain

In [ ]:
#strain = '322b' # L kefiranofaciens
strain = '230a' # L mesenteroides
#strain = '261'  # L lacis

### Load transcriptomics data

In [ ]:
df = pd.read_excel("../transcriptomics/deseq2_results.xlsx", sheet_name=strain)
df["gene"] = df["Unnamed: 0"].apply(lambda x:  "_".join(x.split("_")[1:]))
df.drop(columns=["Unnamed: 0", "baseMean", "lfcSE", "stat", "pvalue", "color", "names"], inplace=True)

### Load eggnog annotations

In [ ]:
df2 = pd.read_csv(f"../eggnog_annotations/eggnog_{strain}.tsv", sep="\t", comment='#', header=None, 
                  usecols=[0,5,6,7,9,11,14,21], 
                  names=["gene", "name", "GO", "EC", "KO", "reaction", "TC", "Description"])

In [ ]:
df3 = pd.merge(df, df2, on="gene")

### Extract GO terms

In [ ]:
entries = []
for _, row in df3.dropna(subset=["GO"]).iterrows():
    for rxn in row["GO"].split(","):
        entries.append((rxn, row["log2FoldChange"], row["padj"]))
                       
df4 = pd.DataFrame(entries, columns=["GO", "log2fc", "padj"])

In [ ]:
go_terms = obo_parser.GODag("../misc_data/go-basic.obo", load_obsolete=True)

### Perform hypergeometric tests

In [ ]:
M = len(df4)
n_up = len(df4.query("log2fc > 1 and padj < 0.05"))
n_down = len(df4.query("log2fc < -1 and padj < 0.05"))

entries_up = []
entries_down = []

for go in sorted(set(df4["GO"])):
    dfi = df4.query("GO == @go")
    name = go_terms[go].name 
    N = len(dfi)

    k_up = len(dfi.query("log2fc > 1 and padj < 0.05"))
    p_up = hypergeom.sf(k_up-1, M, N, n_up)
    entries_up.append((go, k_up, N, p_up, name))

    k_down = len(dfi.query("log2fc < -1 and padj < 0.05"))
    p_down = hypergeom.sf(k_down-1, M, N, n_down)
    entries_down.append((go, k_down, N, p_down, name))
    
df_up = pd.DataFrame(entries_up, columns=["GO", "k", "n", "p_value", "description"])
df_down = pd.DataFrame(entries_down, columns=["GO", "k", "n", "p_value", "description"])

### Filter significant cases and save results

In [ ]:
df_up = df_up.query("p_value < 0.05").sort_values("p_value")
df_down = df_down.query("p_value < 0.05").sort_values("p_value")

In [ ]:
df_up.to_excel(f"../go_term_enrichment/up_{strain}.xlsx", index=False)
df_down.to_excel(f"../go_term_enrichment/down_{strain}.xlsx", index=False)